# Resume draft_data

### Select Columns

In [1]:
import pandas as pd
import numpy as np

In [2]:
#Read all_decks file and get integer columns
df = pd.read_csv("Data/draft_data_public_BRO_PremierDraft.csv",nrows=1)
#(columns int64 -> int8)
columns = []
for column in df.columns.unique():
    if 'pool_' in column:
        columns.append(column)
    if 'pack_card_' in column:
        columns.append(column)

# Make the dtype dict (int64 -> int8)
conversion_integers_columns = {item: np.int8 for item in columns}
conversion_integers_columns['user_game_win_rate_bucket'] = np.float32
conversion_integers_columns['pick'] = 'category'

### Transform columns

In [3]:
import pandas as pd

best_players_pick_list = []
df = pd.read_csv("Data/draft_data_public_BRO_PremierDraft.csv", chunksize=100000, dtype=conversion_integers_columns)
for chunk in df:
    best_players_pick_list.append(chunk[(chunk['rank'] == 'platinum') | (chunk['rank'] == 'diamond') | (chunk['rank'] == 'mythic')])

In [4]:
# Concat and create new .csv file
best_players_df = pd.concat(best_players_pick_list)
best_players_df.to_csv("Data/draft_data_best_players.csv", index=False)

## Create all_decks_file

In [5]:
best_players_df = pd.read_csv("Data/draft_data_best_players.csv", dtype=conversion_integers_columns)

In [6]:
# Get the last pick of last pack
df_deck = best_players_df[(best_players_df['pack_number'] == 2) & (best_players_df['pick_number'] == 14)]

In [7]:
# Create all_decks.csv file
df_deck.to_csv("Data/all_decks.csv", index=False)

In [8]:
del best_players_df

# Add stop cards on dataframe

In [9]:
import pandas as pd
df_deck = pd.read_csv("Data/all_decks.csv", dtype=conversion_integers_columns)
# Insert a stop word to finish the deck
df_deck['pool_Energy Refractor'].head(1)
df_stops = df_deck.copy(deep=True)
df_stops.head(1)

#Zera as cartas do pack atual (nenhum pack foi aberto)
df_stops.loc[:, df_stops.columns.str.startswith('pack_card_')] = 0

#Adicionar a ultima carta aberta na pool do deck
picked_cards = df_stops['pick'].unique()
# Adicionar ultimo pick (#15) como palavra chave: "last"
df_stops['pick_number'] = 15
df_stops['pick'] = 'last'
for card in picked_cards:
    df_stops.loc[df_stops['pick'] == card,'pool_'+card] = df_stops[df_stops['pick'] == card]['pool_'+card] + 1

del df_deck

In [10]:
df_stops.to_csv("Data/df_stops.csv",index=False)

In [11]:
# Load Entire Dataframe
best_players_pick_df = pd.read_csv("Data/draft_data_best_players.csv", dtype=conversion_integers_columns)
# Add Stops Values on big Dataframe
best_players_pick_df = pd.concat([best_players_pick_df,df_stops])
# To csv (with stops cards)
best_players_pick_df.to_csv("Data/draft_data_best_players_sc.csv",index=False)

In [12]:
del best_players_pick_df
del df_stops

## Update all_decks_file with all rows as last pick

In [14]:
best_players_df = pd.read_csv("Data/draft_data_best_players_sc.csv", dtype=conversion_integers_columns)

In [15]:
# Get the last pick of last pack
df_deck = best_players_df[(best_players_df['pack_number'] == 2) & (best_players_df['pick_number'] == 15)]

In [16]:
# Create all_decks.csv file
df_deck.to_csv("Data/all_decks.csv", index=False)
del df_deck